In [20]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="StIIME0OyhYqSYiNLqKl")
project = rf.workspace().project("pill-tdcf6")
dataset = project.version(2).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to pill-2 in yolov5pytorch:: 100%|██████████| 206/206 [00:00<00:00, 1244.17it/s]


In [5]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 10395, done.
remote: Total 10395 (delta 0), reused 0 (delta 0), pack-reused 10395
Receiving objects: 100% (10395/10395), 10.56 MiB | 13.27 MiB/s, done.
Resolving deltas: 100% (7186/7186), done.


In [6]:
%cd /content/yolov5
!pip install -r requirements.txt

/content/yolov5


In [7]:
%cat /content/dataset/data.yaml


names:
- pill
nc: 1
train: pill-1/train/images
val: pill-1/valid/images


In [21]:
%cd /
from glob import glob

img_list = glob('/content/dataset/train/images/*.jpg')

print(len(img_list))

/
100


In [22]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list))

80


In [23]:
with open('/content/dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/dataset/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [24]:
import yaml

with open('/content/dataset/data.yaml', 'r') as f:
    data = yaml.load(f, Loader=yaml.FullLoader)

print(data)

data['train'] = '/content/dataset/train.txt'
data['val'] = '/content/dataset/val.txt'

with open('/content/dataset/data.yaml', 'w') as f:
    yaml.dump(data, f)

print(data)

{'names': ['pill'], 'nc': 1, 'train': 'pill-2/train/images', 'val': 'pill-2/valid/images'}
{'names': ['pill'], 'nc': 1, 'train': '/content/dataset/train.txt', 'val': '/content/dataset/val.txt'}


In [25]:
%cd /content/yolov5/

!python train.py --img 416 --batch 16 --epochs 50 --data /content/dataset/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name pill_yolov5s_results

/content/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/dataset/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=pill_yolov5s_results, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-166-g968e300 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_

In [42]:
from IPython.display import Image
import os

val_img_path = val_img_list[5]

!python detect.py --weights /content/yolov5/runs/train/pill_yolov5s_results3/weights/best.pt --img 416 --conf 0.5 --source "{val_img_path}"

Image(os.path.join('/content/yolov5/inference/output', os.path.basename(val_img_path)))

detect: weights=['/content/yolov5/runs/train/pill_yolov5s_results3/weights/best.pt'], source=/content/dataset/train/images/57_jpg.rf.571b10689de2a4e34d68411e1d3ea65c.jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-166-g968e300 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/dataset/train/images/57_jpg.rf.571b10689de2a4e34d68411e1d3ea65c.jpg: 416x416 2 pills, Done. (0.029s)
Speed: 0.4ms pre-process, 28.7ms inference, 1.8ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp6


In [36]:
%rm -rf /content/yolov5/runs/detect/*
